# Delta Lake Introduction

## Load the data from Volume ecommerce_data from ecommerce DB

In [0]:
# Load data
events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header=True, inferSchema=True)

## Convert to Delta

In [0]:
events.write.format("delta").mode("overwrite").save("/Volumes/workspace/ecommerce/ecommerce_data/events")

## Create managed table

In [0]:
events.write.mode("overwrite").format("delta").saveAsTable("event_table")

## SQL approach

In [0]:
spark.sql("""
    CREATE TABLE events_delta
    USING DELTA
    AS SELECT * FROM event_table
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
# Test schema enforcement
try:
    wrong_schema = spark.createDataFrame([("a","b","c")], ["x","y","z"])
    wrong_schema.write.format("delta").mode("append").save("/Volumes/workspace/ecommerce/ecommerce_data/events")

except Exception as e:
    print(f"Schema enforcement: {e}")

Schema enforcement: [_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: b21a1d8b-8362-4ead-8b8d-6863449f6712).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- event_time: timestamp (nullable = true)
-- event_type: string (nullable = true)
-- product_id: integer (nullable = true)
-- category_id: long (nullable = true)
-- category_code: string (nullable = true)
-- brand: string (nullable = true)
-- price: double (nullable = true)
-- user_id: integer (nullable = true)
-- user_session: string (nullable = true)


Data schema:
root
-- x: string (nullable = true)
-- y: string (nullable = true)
-- z: string (nullable = true)

         
Table ACLs are enabled in this cluster, so automati